## LSSTCam sky coverage
Scroll through on sky images to see coverage
Craig Lage - 23-Apr-25

In [ ]:
import os
import shlex, subprocess
import matplotlib.pyplot as plt
from lsst.summit.utils.plotting import plot
import numpy as np
from astropy.io import fits
from lsst.resources import ResourcePath
from astropy.time import Time, TimeDelta
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.daf.butler import Butler


# Scroll through historical data to see which
# AzEl values have been covered

In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation'])


In [ ]:
startDay = 20250416
endDay = 20250422

azs = []
els = []
dayObs = startDay
while dayObs <= endDay:
    records = butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs)
    print(dayObs, records.count())
    if records.count() < 200:
        dayObs = calcNextDay(dayObs)  # find the next dayObs                                                                                       
        continue
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
        if record.observation_type in ['acq', 'science'] and record.exposure_time > 2.0:
            #print("Adding", record.id)
            exposureList.append(record.id)
    exposureList = sorted(exposureList)
    for expId in exposureList:
        try:
            mData = butler.get('raw.metadata', exposure=expId, detector=94, instrument='LSSTCam')
            az = mData['AZSTART']
            el = mData['ELSTART']
            azs.append(az)
            els.append(el)
            #print(expId, az, el)
        except:
            print(f"{expId} failed!")
            continue
    print(f"Done with {dayObs}")
    print()
    dayObs = calcNextDay(dayObs)  # find the next dayObs 


In [ ]:
plt.scatter(azs, els)
plt.xlim(-180, 180)
plt.ylim(0, 90)